# **Deep Learning Classifier**

In [1]:
import sys
sys.path.append('../src')
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from embedding_hf import encode_texts
from models import Classifier
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pickle



c:\Users\paula\github-classroom\Self-Service_Portal_Final_Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **Load CSV and generate embeddings**
 Convert phrases in embeddings using same hugging face.

Input text list  (["How do I pay?", "I’m stressed", ...])

Output: Matrix NumPy with vectors of 384 dim (X.shape = (60, 384)) --> 60 phrases

In [2]:
# Load trainning queryes
df = pd.read_csv("../data/raw/training_queries.csv")
texts = df["text"].tolist()
labels = df["label"].tolist()

# Transform into embeddings
X = encode_texts(texts)  # numpy array


c:\Users\paula\github-classroom\Self-Service_Portal_Final_Project\venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## **Encode labels**

Using LabelEncoder to convert text to number, then to tensor

In [3]:
le = LabelEncoder()
y_encoded = le.fit_transform(labels)  # convert to [0, 1, 2]
y = torch.tensor(y_encoded, dtype=torch.long)

## **Create Dataset and DataLoader**

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


class QueryDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = QueryDataset(X_train, y_train)
val_dataset = QueryDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


## **Defining MLP model**

1. Input Layer 384 dim (Dense (Linear))
2. Relu (activation function)
2. Dropout avoid overfitting - Regularization
3. Lineal layer

In [5]:
# class Classifier(nn.Module):
#     def __init__(self, input_dim=384, hidden_dim=128, output_dim=3):
#         super().__init__()
#         self.fc1 = nn.Linear(input_dim, hidden_dim)
#         self.dropout = nn.Dropout(0.3)
#         self.fc2 = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         return self.fc2(x)

model = Classifier()

## **Training & Saving the model**

we setup early stopper to save the best model

In [6]:
best_val_loss = float('inf')
epochs = 20
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
patience = 3  # before to stop if does not improve
patience_counter = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total
    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {total_loss:.4f} | Val Loss: {val_loss:.4f} | Accuracy: {accuracy:.2%}")

    # EARLY STOPPING
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "../models/best_query_classifier.pt")  # Saving the best model
        # torch.save(model, "../models/best_query_classifierFULL.pt")

    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"⏹️ Early stopping at epoch {epoch+1}. Best val loss: {best_val_loss:.4f}")
            break


Epoch 1/20 - Train Loss: 13.7284 | Val Loss: 4.0584 | Accuracy: 63.16%
Epoch 2/20 - Train Loss: 13.0515 | Val Loss: 3.9067 | Accuracy: 78.95%
Epoch 3/20 - Train Loss: 12.2884 | Val Loss: 3.6808 | Accuracy: 89.47%
Epoch 4/20 - Train Loss: 11.2064 | Val Loss: 3.3885 | Accuracy: 89.47%
Epoch 5/20 - Train Loss: 9.7377 | Val Loss: 3.0572 | Accuracy: 94.74%
Epoch 6/20 - Train Loss: 8.2233 | Val Loss: 2.7126 | Accuracy: 94.74%
Epoch 7/20 - Train Loss: 6.9473 | Val Loss: 2.3870 | Accuracy: 100.00%
Epoch 8/20 - Train Loss: 5.9228 | Val Loss: 2.0577 | Accuracy: 100.00%
Epoch 9/20 - Train Loss: 4.6073 | Val Loss: 1.7772 | Accuracy: 100.00%
Epoch 10/20 - Train Loss: 3.7979 | Val Loss: 1.5457 | Accuracy: 100.00%
Epoch 11/20 - Train Loss: 3.0570 | Val Loss: 1.3518 | Accuracy: 100.00%
Epoch 12/20 - Train Loss: 2.5391 | Val Loss: 1.1801 | Accuracy: 100.00%
Epoch 13/20 - Train Loss: 2.0307 | Val Loss: 1.0337 | Accuracy: 100.00%
Epoch 14/20 - Train Loss: 1.7842 | Val Loss: 0.9202 | Accuracy: 100.00%
Epo

## **Saving the model**

In [7]:
with open("../models/label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

print("Saving Encode")


Saving Encode


In [8]:
def classify_query(text):
    embedding = encode_texts([text])  # np array con shape (1, 384)
    tensor = torch.tensor(embedding, dtype=torch.float32)
    with torch.no_grad():
        logits = model(tensor)
        pred = torch.argmax(logits, dim=1).item()
    return le.inverse_transform([pred])[0]

examples = [
    "How do I make a payment?",
    "Where can I access VMock?",
    "Nothing is working and I'm tired.",
    "I need help with OSAP documents.",
    "Can someone please help me? I'm so confused.",
    "Hola",
    "hi again"
]

for text in examples:
    print(f"{text} → {classify_query(text)}")

How do I make a payment? → faq
Where can I access VMock? → faq
Nothing is working and I'm tired. → offramp
I need help with OSAP documents. → resource
Can someone please help me? I'm so confused. → offramp
Hola → chitchat
hi again → chitchat


c:\Users\paula\github-classroom\Self-Service_Portal_Final_Project\venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\paula\github-classroom\Self-Service_Portal_Final_Project\venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\paula\github-classroom\Self-Service_Portal_Final_Project\venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\paula\github-classroom\Self-Service_Portal_Final_Project\venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWa